In [1]:
# %pip install mayavi
# %pip install PyQt5
# %pip install mayavi nibabel
# %pip install nilearn

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab
from allensdk.core.cell_types_cache import CellTypesCache
import PyQt5.QtWidgets as qt
import nibabel as nib
import nilearn.plotting as plotting
import os

In [18]:
# Check if cells.json exists
if not os.path.exists('cell_types/cells.json'):
    # Download the 3D reconstruction data using the AllenSDK
    ctc = CellTypesCache(manifest_file='cell_types/manifest.json')
    # a list of cell metadata for cells with reconstructions, download if necessary
    cells = ctc.get_cells(require_reconstruction=True)

In [4]:
# read the reconstruction from the first 100 cells
reconstructions = []
for cell in cells[0:5]:
    print(cell['id'])
    reconstructions.append(ctc.get_reconstruction(cell['id']))

485909730
323865917
476135066
502614426
519832676


In [19]:
# get location of the soma
reconstructions[0].soma.keys()

dict_keys(['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'tree_id', 'children'])

In [20]:
# Extract the 3D coordinates of each neuron's soma
soma_locations = []
for reconstruction in reconstructions:
    soma_location = (
        reconstruction.soma['x'],
        reconstruction.soma['y'],
        reconstruction.soma['z']
    )
    soma_locations.append(soma_location)
    print(soma_location)

# Create a Mayavi figure
fig = mlab.figure(size=(800, 500))

# Load the brain data using nibabel
brain_data = nib.load('../data/MNI152_T1_1mm_brain.nii.gz')
data_array = brain_data.get_fdata()

# Center the brain data around the origin of the coordinate system
centered_data_array = data_array - np.mean(data_array)

# Create a 3D volume rendering of the brain data in the center
source = mlab.pipeline.scalar_field(centered_data_array)

# Create a 3D mesh of the brain
mlab.pipeline.iso_surface(
    source, contours=[1000, ], opacity=0.3, colormap='gray')

# Convert the soma locations to be relative to the brain location
soma_locations = np.array(soma_locations)

# Plot the soma locations as spheres
for i in range(len(soma_locations)):
    mlab.points3d(
        soma_locations[i, 0],
        soma_locations[i, 1],
        soma_locations[i, 2],
        scale_factor=20,
        color=(0, 0, 1)
    )

# Plot the center of the coordinate system (origin)
mlab.points3d(0, 0, 0, scale_factor=10, color=(1, 0, 0), mode='sphere')

# Plot the 3 axes
mlab.outline()

# Display the visualization
mlab.show()

(375.2331, 548.5594, 19.3472)
(280.3944, 575.6608, 30.8)
(405.6624, 431.7456, 21.84)
(471.4424, 431.0592, 26.32)
(475.8113, 605.4426, 51.0462)


In [7]:
print(data_array.shape)

(182, 218, 182)
